# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
output_data_file = "output_data/cities.csv"

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.82,73,80,13.13,TF,1699761510
1,1,stanley,54.8680,-1.6985,4.60,100,40,1.03,GB,1699761334
2,2,debno,52.7390,14.6980,5.64,91,99,2.22,PL,1699761511
3,3,la passe,45.5549,-0.8967,15.13,83,100,11.44,FR,1699761511
4,4,kirakira,-10.4544,161.9205,27.62,82,100,4.17,SB,1699761511


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display
humidity = city_data_df["Humidity"].astype(int)
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
   "Lng",
   "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size ="Humidity",
    color = "City",
    alpha = .75
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()


# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,acapulco de juarez,16.8634,-99.8901,26.55,84,0,1.91,MX,1699761271
166,166,pokhara,28.2333,83.9833,21.15,52,0,1.69,NP,1699761538
197,197,medina,24.4686,39.6142,22.24,30,0,1.03,SA,1699761544
254,254,belo sur tsiribihina,-19.7000,44.5500,26.66,59,0,1.28,MG,1699761559
318,318,kidal,18.4411,1.4078,23.60,11,0,4.22,ML,1699761574
339,339,pamandzi,-12.7967,45.2794,26.88,78,0,2.57,YT,1699761579
429,429,la palma,33.8464,-118.0467,22.28,41,0,0.00,US,1699761594
504,504,howrah,22.5892,88.3103,26.00,73,0,1.54,IN,1699761628
514,514,al burayqah,30.4062,19.5739,22.41,49,0,4.17,LY,1699761637
535,535,tura,25.5198,90.2201,24.18,62,0,3.64,IN,1699761653


In [11]:
city_data_df.count()

City_ID       10
City          10
Lat           10
Lng           10
Max Temp      10
Humidity      10
Cloudiness    10
Wind Speed    10
Country       10
Date          10
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df[["City", "Country", "Max Temp", "Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
55,acapulco de juarez,MX,26.55,16.8634,-99.8901,84,
166,pokhara,NP,21.15,28.2333,83.9833,52,
197,medina,SA,22.24,24.4686,39.6142,30,
254,belo sur tsiribihina,MG,26.66,-19.7000,44.5500,59,
318,kidal,ML,23.60,18.4411,1.4078,11,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "types" : "lodging",
    "Api_key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon}{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
acapulco de juarez - nearest hotel: No hotel found
pokhara - nearest hotel: No hotel found
medina - nearest hotel: No hotel found
belo sur tsiribihina - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
pamandzi - nearest hotel: No hotel found
la palma - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
al burayqah - nearest hotel: No hotel found
tura - nearest hotel: No hotel found


,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
55,acapulco de juarez,MX,26.55,16.8634,-99.8901,84,No hotel found
166,pokhara,NP,21.15,28.2333,83.9833,52,No hotel found
197,medina,SA,22.24,24.4686,39.6142,30,No hotel found
254,belo sur tsiribihina,MG,26.66,-19.7000,44.5500,59,No hotel found
318,kidal,ML,23.60,18.4411,1.4078,11,No hotel found
339,pamandzi,YT,26.88,-12.7967,45.2794,78,No hotel found
429,la palma,US,22.28,33.8464,-118.0467,41,No hotel found
504,howrah,IN,26.00,22.5892,88.3103,73,No hotel found
514,al burayqah,LY,22.41,30.4062,19.5739,49,No hotel found
535,tura,IN,24.18,25.5198,90.2201,62,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)


# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)